# Task
Fine tune the Qwen/Qwen3-8B model from Hugging Face to act as a Lean 4 Prover using the FrenzyMath/Herald_proofs dataset from Hugging Face, applying best practices for fine-tuning.

## Set up the environment

### Subtask:
Install necessary libraries and dependencies, such as `transformers`, `datasets`, `peft`, and `torch`.


**Reasoning**:
Install the required libraries using pip.



In [1]:
!pip install -U trl transformers accelerate peft datasets bitsandbytes torch triton protobuf 

## Load and preprocess the dataset

### Subtask:
Load the FrenzyMath/Herald_proofs dataset from Hugging Face. The dataset needs to be preprocessed to format the data into suitable input and output pairs for the model. This might involve converting the informal statements and Lean 4 proofs into a format the model can understand.


**Reasoning**:
Load the dataset and inspect its structure.



In [2]:
from datasets import load_dataset

jsonl_file = "final_dataset.jsonl"

dataset = load_dataset("json", data_files=jsonl_file, split='train')

print("First Dataset:")
print(dataset)
print("\n Example first data")
print(dataset[0])


split_dataset = dataset.train_test_split(test_size=0.2, seed=42)

print("\nDataset have been splitted")
print(split_dataset)

/home/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First Dataset:
Dataset({
    features: ['prompt', 'chosen', 'rejected', 'chosen_efficiency', 'rejected_efficiency', 'efficiency_difference', 'category', 'variation', 'group_key', 'pair_type'],
    num_rows: 4444
})

 Example first data
{'prompt': "[USER] \nGenerate unique Python code to learn new information about the network environment (your current container or accessible devices).\nHere's are some data about the environment:\n<EnvInfos>\nContainer ID: {containerID}\nRead/Write Size: 24015.1591796875 KB\nFree Disk Space (v1): 1487676.0 KB\nFree Disk Space (v2): 1487676.0 KB\n\nContainer ID: {containerID}\nRead/Write Size: 23750.001953125 KB\nFree Disk Space (v1): 1487676.0 KB\nFree Disk Space (v2): 1487676.0 KB\n\n</EnvInfos>\n<SpecialEnvInfo>\nHostname: {containerID}, IP Address: 172.19.0.2\n\nname 'shutil' is not defined\n\n</SpecialEnvInfo>", 'chosen': '```python\nimport subprocess\n\ndef get_network_info():\n    try:\n        # Get container ID\n        container_id = subprocess

**Reasoning**:
Define a preprocessing function to format the data and apply it to the dataset.



In [3]:
def preprocess_function(examples):
    return {
        "prompt": [str(p) for p in examples["prompt"]],
        "chosen": [str(c) for c in examples["chosen"]],
        "rejected": [str(r) for r in examples["rejected"]],
    }

from datasets import DatasetDict
split_dataset_dict = DatasetDict(split_dataset)

processed_dataset = split_dataset_dict.map(
    preprocess_function,
    batched=True,
    remove_columns=list(split_dataset_dict["train"].features)
)

print("\n Dataset after proccessing:")
print(processed_dataset)
print("\n Example date:")
print(processed_dataset['train'][0])


 Dataset after proccessing:
DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 3555
    })
    test: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 889
    })
})

 Example date:
{'prompt': '[USER] \nGenerate unique Python code to learn new information about the network environment (your current container or accessible devices).\nHere\'s are some data about the environment:\n<EnvInfos>\nContainer ID: {containerID}\nRead/Write Size: 23710.11328125 KB\nFree Disk Space (v1): 1229504.0 KB\nFree Disk Space (v2): 1229504.0 KB\n\nContainer ID: {containerID}\nRead/Write Size: 23667.001953125 KB\nFree Disk Space (v1): 1229504.0 KB\nFree Disk Space (v2): 1229504.0 KB\n\n</EnvInfos>\n<SpecialEnvInfo>\n{\n    "networks": "[Errno 2] No such file or directory: \'docker\'",\n    "containers": "[Errno 2] No such file or directory: \'docker\'",\n    "topology": "[Errno 2] No such file or directory: \'docker\'"\n}\n\n

**Reasoning**:
Split the preprocessed dataset into training and evaluation sets.



In [ ]:
# DONT RUN THIS


train_test_split = processed_dataset['train'].train_test_split(test_size=0.2)
dataset['train'] = train_test_split['train']
dataset['test'] = train_test_split['test']
print(dataset)

## Load the base model and tokenizer

### Subtask:
Load the Qwen/Qwen3-8B model and its corresponding tokenizer from Hugging Face.


**Reasoning**:
Load the pre-trained Qwen/Qwen3-8B model and tokenizer from Hugging Face.



In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments

# --- 1. Configure 4-bit Quantization ---
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
local_tokenizer_path = "/home/GA/tokenizer" 


tokenizer = AutoTokenizer.from_pretrained(
    local_tokenizer_path,
    trust_remote_code=True
)

# Important: Add a padding token if the tokenizer doesn't have one
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- 2. Load the model with the quantization config ---
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto", # Automatically map model to GPU
    trust_remote_code=True,
    pad_token_id=tokenizer.pad_token_id
)

model.config.pad_token_id = tokenizer.pad_token_id

print(model)
print(tokenizer)

print(f"Tokenizer Pad Token: '{tokenizer.pad_token}' | ID: {tokenizer.pad_token_id}")
print(f"Model Config Pad Token ID: {model.config.pad_token_id}")


Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.74s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584, padding_idx=151643)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm): Qw

## Configure for lora fine-tuning

### Subtask:
Set up the model for fine-tuning using the LoRA (Low-Rank Adaptation) technique, which is a parameter-efficient method suitable for large models. This involves configuring the LoRA parameters.


**Reasoning**:
Set up the LoRA configuration and apply it to the model, then print the trainable parameters.



In [6]:
# from peft import LoraConfig, get_peft_model

# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

# model = get_peft_model(model, lora_config)

# model.print_trainable_parameters()

# from peft import prepare_model_for_kbit_training

# print("Preparing model for k-bit training...")
# model = prepare_model_for_kbit_training(model)


# NOW, CONFIGURE AND APPLY LORA
# from peft import LoraConfig, get_peft_model

from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

print("Preparing model for k-bit training...")
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

print("Applying LoRA config to the model...")
model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

Preparing model for k-bit training...
Applying LoRA config to the model...
trainable params: 10,092,544 || all params: 7,625,709,056 || trainable%: 0.1323


## Set up training arguments

### Subtask:
Define the training parameters, such as learning rate, batch size, number of epochs, and other relevant settings for the fine-tuning process.


**Reasoning**:
Define the training arguments for the fine-tuning process.



In [7]:
from trl import DPOConfig

# training_args = TrainingArguments(
#     output_dir="./qwen-lean4-prover-finetuned",
#     num_train_epochs=3,
#     per_device_train_batch_size=1,  # Use a small batch size
#     gradient_accumulation_steps=8, # Accumulate gradients over 8 steps
#     # Effective batch size is 1 * 8 = 8
#     warmup_steps=500,
#     weight_decay=0.01,
#     learning_rate=2e-5,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_total_limit=2,
#     save_steps=500,
#     bf16=True, # Enable bf16 for more memory savings and speed
# )

training_args = DPOConfig(
    output_dir="./GA-QWEN2.5-Coder-7B-Instruct_DPO",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=500,
    bf16=True,
    optim="paged_adamw_8bit", # Use the 8-bit paged optimizer
    gradient_checkpointing=True, # Further save memory at the cost of speed
    # eval_strategy="steps",
    # eval_steps=500,
)

print(training_args)

DPOConfig(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
base_model_attribute_name=model,
batch_eval_metrics=False,
beta=0.1,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
dataset_num_proc=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_dropout=True,
disable_tqdm=False,
discopop_tau=0.05,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_

## Start training

### Subtask:
Begin the fine-tuning process using the `trainer.train()` method.


**Reasoning**:
Call the train() method on the trainer object to initiate the fine-tuning process.



In [8]:
from trl import DPOTrainer

trainer = DPOTrainer(
    model=model,
    args=training_args,
    # beta=0.1,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    processing_class=tokenizer,
    # max_prompt_length=1024,
    # max_length=2048,
    # max_target_length=1024 
)

print("\n--- Start ---")
trainer.train()
print("\n--- Completed ---")

Tokenizing eval dataset: 100%|██████████| 889/889 [00:11<00:00, 76.26 examples/s] 
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 151643, 'bos_token_id': None}.



--- Start ---


/home/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,0.671400
20,0.575300
30,0.447500
40,0.331800
50,0.287500
60,0.227500
70,0.210800
80,0.173100
90,0.150300
100,0.164500


/home/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/home/.venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



--- Completed ---


## Save the fine-tuned model

### Subtask:
After training is complete, save the fine-tuned model and tokenizer.


**Reasoning**:
Save the fine-tuned model and tokenizer using the trainer's save_model method and verify the files are saved.



In [9]:
save_directory = "./GA-QWEN2.5-Coder-7B_DPO_Instruct-Model" #GANTI NAMA 
trainer.save_model(save_directory)

import os

# Verify that the model and tokenizer files have been saved
model_files = [f for f in os.listdir(save_directory) if os.path.isfile(os.path.join(save_directory, f))]
print("Files saved in the model directory:")
for file in model_files:
    print(file)

tokenizer_files = [f for f in os.listdir(save_directory) if os.path.isfile(os.path.join(save_directory, f))]
print("\nFiles saved in the tokenizer directory (should be the same as model directory):")
for file in tokenizer_files:
    print(file)

Files saved in the model directory:
adapter_model.safetensors
merges.txt
adapter_config.json
special_tokens_map.json
vocab.json
training_args.bin
tokenizer_config.json
chat_template.jinja
README.md
added_tokens.json
tokenizer.json

Files saved in the tokenizer directory (should be the same as model directory):
adapter_model.safetensors
merges.txt
adapter_config.json
special_tokens_map.json
vocab.json
training_args.bin
tokenizer_config.json
chat_template.jinja
README.md
added_tokens.json
tokenizer.json


**Reasoning**:
The previous attempt failed because the `trainer` object was not defined in the current session. Re-instantiate the `Trainer` and then save the model and tokenizer.



In [10]:
%pip install huggingface_hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install ipywidgets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 174.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]
Note: you may need to restart the kernel to use updated packages.


In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# --- 1. Define paths and the new repository ID ---
base_model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
adapter_path = "./GA-QWEN2.5-Coder-7B-Instruct_DPO/checkpoint-1335" # Update with your checkpoint path
repo_id = "ClickNoow/ga-qwen2.5-Coder-7B-Instruct-finetuned-DPO" # CHANGE to your username

# --- 2. Load the model and tokenizer from your local files ---
# (The login is now handled automatically by the token you saved in the terminal)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")
model = PeftModel.from_pretrained(base_model, adapter_path)
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

# --- 3. Push everything to the Hub ---
print(f"Pushing model and tokenizer to {repo_id}...")
model.push_to_hub(repo_id)
tokenizer.push_to_hub(repo_id)

print("✅ Upload complete!")

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.44it/s]


Pushing model and tokenizer to ClickNoow/ga-qwen2.5-Coder-7B-Instruct-finetuned-DPO...


Processing Files (1 / 1): 100%|██████████| 40.4MB / 40.4MB, 10.1MB/s  
New Data Upload: 100%|██████████| 40.4MB / 40.4MB, 10.1MB/s  
Processing Files (1 / 1): 100%|██████████| 11.4MB / 11.4MB,  548kB/s  
New Data Upload: 100%|██████████|  329kB /  329kB,  548kB/s  


✅ Upload complete!


In [ ]:
save_directory = "./fine_tuned_qwen_lean4"
# Replace "your_username/your_model_name" with your desired repository name
repo_name = "korazer/fine_tuned_qwen_lean4"

trainer.push_to_hub(repo_name)